# Black-Litterman Analysis on Eurostat Data
Ellie Cox

This file will conduct markowitz and black-litterman analysis on energy supply data retrieved from eurostat. 
First with the full data, then taking the average when excluding one country at a time, the average when excluding one year of data at a time, and lastly using a randomly selected 75\% of the data

## Load Packages

In [3]:
import numpy as np
import pandas as pd
import scipy.stats as stat
import matplotlib.pyplot as plt
from pylab import rcParams
%matplotlib inline

In [4]:
import pypfopt as pyp
import warnings
warnings.filterwarnings("ignore")

## Read Data

In [5]:
# Read data using pandas
data = pd.read_csv("/Users/elizabeth/Documents/Master's Project/Data/EU_TotalEnergySupply.csv")

# Create list of column names
data.columns.values.tolist()
# Rename columns to make life easier
data.columns = [c.replace(' ', '_') for c in data.columns] # remove spaces
data.columns = [c.replace('(', '') for c in data.columns] # remove open parenthesis
data.columns = [c.replace(')', '') for c in data.columns] # remove close parenthesis
data.columns.values.tolist()

# Get rid of ':' and shorten other names
data = data.replace([':'],'')
data = data.replace(['European Union - 27 countries (from 2020)'],'EU')
data = data.replace(['Euro area - 19 countries  (from 2015)'],'Euro area')
data = data.replace(['Germany (until 1990 former territory of the FRG)'],'Germany')
data = data.replace(['Kosovo (under United Nations Security Council Resolution 1244/99)'],'Kosovo')

# Change Data type to numeric
data[data.columns[2:]] = data[data.columns[2:]].apply(pd.to_numeric, errors ='coerce')

## Missing Data
Missing Data is handled in 2 ways:

    1) Replacing missing data with the country's average
    2) Dropping it

## Impute Data

In [6]:
## 1) Impute data with the average
frames = []
for i in list(set(data['Country'])):
            df_country = data[data['Country'] == i] 
            df_country['Total_GWH'].fillna(df_country['Total_GWH'].mean(),inplace = True)
            df_country['Solid_fossil_fuels'].fillna(df_country['Solid_fossil_fuels'].mean(), inplace = True)
            df_country['Peat_and_peat_products'].fillna(df_country['Peat_and_peat_products'].mean(), inplace = True)
            df_country['Solar_Thermal'].fillna(df_country['Solar_Thermal'].mean(), inplace = True)
            df_country['Oil_and_petroleum_products'].fillna(df_country['Oil_and_petroleum_products'].mean(), inplace = True)
            df_country['Natural_gas'].fillna(df_country['Natural_gas'].mean(), inplace = True)
            df_country['Renewables_and_biofuels'].fillna(df_country['Renewables_and_biofuels'].mean(), inplace = True)
            df_country['Nuclear_heat'].fillna(df_country['Nuclear_heat'].mean(),inplace = True)
            df_country['Hydro'].fillna(df_country['Hydro'].mean(),inplace = True)
            df_country['Geothermal'].fillna(df_country['Geothermal'].mean(),inplace = True)
            df_country['Ambient_Heat'].fillna(df_country['Ambient_Heat'].mean(),inplace = True)
            df_country['Tide_wave_and_ocean'].fillna(df_country['Tide_wave_and_ocean'].mean(),inplace = True)
            df_country['Wind'].fillna(df_country['Wind'].mean(),inplace = True)
            df_country['Biofuels_solid'].fillna(df_country['Biofuels_solid'].mean(),inplace = True)
            df_country['Biofuels_other'].fillna(df_country['Biofuels_other'].mean(),inplace = True)
            df_country['Biofuels'].fillna(df_country['Biofuels'].mean(),inplace = True)
            frames.append(df_country)
            final_df = pd.concat(frames)
data_impute = final_df

## Define B-L Function

In [30]:
def bl(n_assets, n_obs, return_vec, view):
    '''
    This function evaluates the equillibrium returns of a portfolio and generates the sample
    Inputs: 
    n_assets: Number of assets
    n_obs: Number of observations
    return_vec: A matrix of returns of shape n_obs x n_assets (a np.array)
    view: a vector of investor views of length n_assets
    Returns: 
    weights: optimal weights
    bl_returns: BL returns
    S: BL risk
    '''
    R = pd.Series(np.mean(return_vec, axis = 0)) # Market returns
    pi = R - 0 * np.ones(n_assets) # equillibrium risk premiums where R_f = 0
    r = np.cov(return_vec.T)
    
    rng = np.random.default_rng()
    cov_struct = rng.multivariate_normal(np.zeros(n_assets), cov = r, size = n_obs)
    S = pyp.risk_models.CovarianceShrinkage(cov_struct).ledoit_wolf()
    delta = pyp.black_litterman.market_implied_risk_aversion(pi, risk_free_rate=0.5)
    
    market_prior = pd.Series(np.mean(return_vec, axis = 0))
    views = pd.Series(view)
    
    bl = pyp.BlackLittermanModel(S, pi=market_prior, absolute_views=views)
    bl_return = bl.bl_returns()

    ef = pyp.EfficientFrontier(bl_return, r)
    bl.bl_weights(delta)
    weights = bl.clean_weights()

    S_bl = bl.bl_cov()
    return weights, bl_return, S_bl

## Optimization on Full Data set

In [52]:
## Create subset of data

cdat = data_impute.loc[:,['Solid_fossil_fuels','Peat_and_peat_products','Oil_and_petroleum_products', 
                  'Natural_gas', 'Nuclear_heat', 'Renewables_and_biofuels']]

In [55]:
## Run one optimization - this returns the results for the full data set and will report the 
## results of one optimization instead of an average 

## Set view - I'm going to start by using the median
view = np.array(np.median(cdat, axis = 0))

full_wt, full_rt, full_rsk = bl(cdat.shape[1], cdat.shape[0], np.array(cdat), view)
weight_full = list(full_wt.items())
w_full = [x[1] for x in weight_full]
full_wt = w_full

In [62]:
print('weights = '+str(full_wt),
      '\nreturns = '+str(np.array(full_rt).T),
      '\nrisks ='+str(np.diagonal(np.matrix(full_rsk))))

weights = [0.15641, 0.0014, 0.33644, 0.23665, 0.11909, 0.15002] 
returns = [109502.09953958    978.35123333 235548.34631236 165683.0556866
  83373.66182262 105027.50452499] 
risks =[7033862.17019782 7033862.17019782 7033862.17019782 7033862.17019782
 7033862.17019782 7033862.17019782]


## Calculate average over dropping one country at a time

In [63]:
## Full data set - with country and year for filtering
cdat = data_impute.loc[:,['Country','Year','Solid_fossil_fuels','Peat_and_peat_products','Oil_and_petroleum_products', 
                  'Natural_gas', 'Nuclear_heat', 'Renewables_and_biofuels']]

In [64]:
weight_res = np.zeros((len(data.groupby('Country')),len(cdat.T)-2))
return_res = np.zeros((len(data.groupby('Country')),len(cdat.T)-2))
risks_res = np.zeros((len(data.groupby('Country')),len(cdat.T)-2))

for i in range(len(data.groupby('Country'))):
    country_dat = cdat.loc[(cdat.Country != cdat.Country.unique()[i])]
    country_dat = country_dat.loc[:,['Solid_fossil_fuels','Peat_and_peat_products','Oil_and_petroleum_products', 
                  'Natural_gas', 'Nuclear_heat', 'Renewables_and_biofuels']]
    ## Set view - I'm going to start by using the median
    view = np.array(np.median(country_dat, axis = 0))
    weights, returns, risks = bl(country_dat.shape[1], country_dat.shape[0], np.array(country_dat), view)
    weights = list(weights.items())
    w = [x[1] for x in weights]
    weight_res[i,:] = w
    return_res[i,:] = np.array(returns).T
    risks_res[i,:] = np.diagonal(np.array(risks))
    

In [68]:
print('weights (mean) = '+str(np.mean(weight_res, axis=0)),
      '\nreturns (mean) = '+str(np.mean(return_res, axis=0)),
      '\nrisks (mean) ='+str(np.mean(risks_res, axis=0)))

weights (mean) = [0.15551571 0.00146238 0.33486452 0.23991143 0.11476381 0.1534819 ] 
returns (mean) = [109260.12630077    991.10394678 236383.49340321 165225.41353941
  83356.82894891 105494.82591162] 
risks (mean) =[7956311.59741047 7905891.31538599 7930779.88998607 7909083.2893372
 8098894.83213647 7905075.69836393]


In [67]:
print('weights (std) = '+str(np.std(weight_res, axis=0)),
      '\nreturns (std) = '+str(np.std(return_res, axis=0)),
      '\nrisks (std) ='+str(np.std(risks_res, axis=0)))

weights (std) = [0.01720593 0.00055449 0.03011869 0.01644752 0.01842228 0.01075178] 
returns (std) = [ 6002.65900369   154.97973476 13600.64202144 10147.97361996
  5874.09629043  5894.24746583] 
risks (std) =[22568094.33363041 22578807.14321893 22571142.59375135 22578010.83195563
 22542141.07985709 22579047.80776891]


## Calculate average over dropping one year at a time

In [70]:
weight_res = np.zeros((len(data.groupby('Year')),len(cdat.T)-2))
return_res = np.zeros((len(data.groupby('Year')),len(cdat.T)-2))
risks_res = np.zeros((len(data.groupby('Year')),len(cdat.T)-2))

for i in range(len(data.groupby('Year'))):
    year_dat = cdat.loc[(cdat.Year != cdat.Year.unique()[i])]
    year_dat = year_dat.loc[:,['Solid_fossil_fuels','Peat_and_peat_products','Oil_and_petroleum_products', 
                  'Natural_gas', 'Nuclear_heat', 'Renewables_and_biofuels']]
    ## Set view - I'm going to start by using the median
    view = np.array(np.median(year_dat, axis = 0))
    weights, returns, risks = bl(year_dat.shape[1], year_dat.shape[0], np.array(year_dat), view)
    weights = list(weights.items())
    w = [x[1] for x in weights]
    weight_res[i,:] = w
    return_res[i,:] = np.array(returns).T
    risks_res[i,:] = np.diagonal(np.array(risks))
    

In [71]:
print('weights (std) = '+str(np.std(weight_res, axis=0)),
      '\nreturns (std) = '+str(np.std(return_res, axis=0)),
      '\nrisks (std) ='+str(np.std(risks_res, axis=0)))

weights (std) = [0.0090399  0.00069333 0.0129027  0.03435812 0.04141464 0.01991285] 
returns (std) = [1857.85279504  110.11929823 2182.46942213  963.14058153 7628.31058654
 1195.63385923] 
risks (std) =[1.52428842e+08 1.52429770e+08 1.52417767e+08 1.52401091e+08
 1.52425398e+08 1.52429462e+08]


## Repeat while Dropping Missing Data

In [73]:
data_drop = data.dropna()

In [74]:
# Subset Data
cdat = data_drop.loc[:,['Solid_fossil_fuels','Peat_and_peat_products','Oil_and_petroleum_products', 
                  'Natural_gas', 'Nuclear_heat', 'Renewables_and_biofuels']]

In [75]:
## Run one optimization - this returns the results for the full data set and will report the 
## results of one optimization instead of an average 

## Set view - I'm going to start by using the median
view = np.array(np.median(cdat, axis = 0))

full_wt, full_rt, full_rsk = bl(cdat.shape[1], cdat.shape[0], np.array(cdat), view)
weight_full = list(full_wt.items())
w_full = [x[1] for x in weight_full]
full_wt = w_full

In [76]:
print('weights = '+str(full_wt),
      '\nreturns = '+str(np.array(full_rt).T),
      '\nrisks ='+str(np.diagonal(np.matrix(full_rsk))))

weights = [0.20352, -0.00945, 0.18989, 0.25681, 0.14415, 0.21509] 
returns = [107684.79483215   -405.9393657  232865.37987819 162990.38697399
  81957.95696558 104429.36979218] 
risks =[ 672929.04933339  590075.7794436  1551239.59521365  786968.47448296
  712916.0618647   606445.43234619]
